In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
##dataset 불러오기
#FashionMNIST 데이터 셋을 DataLoder를 부를때 정의
#Transforms를이용한 전처리는 파이토치 텐서화 & 정규화
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train = True,
                         download = True,
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,),(0.3081,))
                         ])),
    batch_size = BATCH_SIZE, shuffle = True
)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                         train = False,
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.1307,),(0.3081,))
                         ])),
    batch_size = BATCH_SIZE, shuffle = True
)

## CNN
컨볼루션 > 풀링 > 컨볼루션 > 드롭아웃 > 풀링 > 신경망 > 드롭아웃 > 신경망

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,10, kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2)) #커널의 크기 = 2 * 2
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2)) # 커널의 크기 = 2 * 2
        x = x.view(-1,320) #320 벡터 #fully-connected 전에 벡터 한줄로
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

In [6]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}"
                 .format(epoch, batch_idx * len(data),
                        len(train_loader.dataset),
                        100.*batch_idx / len(train_loader),
                        loss.item()))

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            #배치 오차률 합산
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            #가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim =True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy  = evaluate(model, test_loader)
    
    print("[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss:2.363700
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.591300
Train Epoch: 1 [25600/60000 (43%)]	Loss:0.990557
Train Epoch: 1 [38400/60000 (64%)]	Loss:0.984466
Train Epoch: 1 [51200/60000 (85%)]	Loss:0.992144
[1] Test Loss: 0.6937, Accuracy: 72.79%
Train Epoch: 2 [0/60000 (0%)]	Loss:0.973777
Train Epoch: 2 [12800/60000 (21%)]	Loss:0.735217
Train Epoch: 2 [25600/60000 (43%)]	Loss:0.871449
Train Epoch: 2 [38400/60000 (64%)]	Loss:0.542240
Train Epoch: 2 [51200/60000 (85%)]	Loss:0.686780
[2] Test Loss: 0.5954, Accuracy: 76.64%
Train Epoch: 3 [0/60000 (0%)]	Loss:0.590347
Train Epoch: 3 [12800/60000 (21%)]	Loss:0.786614
Train Epoch: 3 [25600/60000 (43%)]	Loss:0.563204
Train Epoch: 3 [38400/60000 (64%)]	Loss:0.645375
Train Epoch: 3 [51200/60000 (85%)]	Loss:0.616206
[3] Test Loss: 0.5494, Accuracy: 77.30%
Train Epoch: 4 [0/60000 (0%)]	Loss:0.709978
Train Epoch: 4 [12800/60000 (21%)]	Loss:0.704723
Train Epoch: 4 [25600/60000 (43%)]	Loss:0.745714
Train Epoch: 4 [38

[29] Test Loss: 0.3341, Accuracy: 87.80%
Train Epoch: 30 [0/60000 (0%)]	Loss:0.665965
Train Epoch: 30 [12800/60000 (21%)]	Loss:0.472427
Train Epoch: 30 [25600/60000 (43%)]	Loss:0.276478
Train Epoch: 30 [38400/60000 (64%)]	Loss:0.357384
Train Epoch: 30 [51200/60000 (85%)]	Loss:0.285451
[30] Test Loss: 0.3287, Accuracy: 87.88%
Train Epoch: 31 [0/60000 (0%)]	Loss:0.321500
Train Epoch: 31 [12800/60000 (21%)]	Loss:0.482475
Train Epoch: 31 [25600/60000 (43%)]	Loss:0.546967
Train Epoch: 31 [38400/60000 (64%)]	Loss:0.431120
Train Epoch: 31 [51200/60000 (85%)]	Loss:0.507365
[31] Test Loss: 0.3287, Accuracy: 87.90%
Train Epoch: 32 [0/60000 (0%)]	Loss:0.463694
Train Epoch: 32 [12800/60000 (21%)]	Loss:0.549069
Train Epoch: 32 [25600/60000 (43%)]	Loss:0.359284
Train Epoch: 32 [38400/60000 (64%)]	Loss:0.460024
Train Epoch: 32 [51200/60000 (85%)]	Loss:0.284715
[32] Test Loss: 0.3240, Accuracy: 87.87%
Train Epoch: 33 [0/60000 (0%)]	Loss:0.449304
Train Epoch: 33 [12800/60000 (21%)]	Loss:0.527125
Train 